<a href="https://colab.research.google.com/github/hasanalay/ML/blob/main/wav2vec2_tr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install huggingface_hub

     |████████████████████████████████| 84 kB 1.2 MB/s 


In [ ]:

!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install torchaudio
!pip install jiwer

     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 28.9 MB/s 
     |████████████████████████████████| 596 kB 47.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 50 kB 3.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149879 sha256=592a31f9b64cfeebdc5c9aa20d04d00df15da21d9e910a32fdcebd36021e6038
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from datasets import load_dataset, load_metric, Audio

In [ ]:
common_voice_train = load_dataset("common_voice", "tr", split="train+validation")
common_voice_test = load_dataset("common_voice", "tr", split="test")

Generating train split:   0%|          | 0/1831 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1647 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1647 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/325 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/18685 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/1726 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/tr/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e. Subsequent calls will reuse this data.


Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/tr/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,Proje Haziran ayına kadar tamamlanacak.
1,Cazseverler de kendi paylarına düşeni aldılar.
2,"Para, bütün bölgesel yönetmenlerin ortak sorunu."
3,Rus Casusu?
4,Onları savaş suçu davalarına nasıl hazırlıyorsunuz?
5,Yumurta aklarını bir çimdik tuzla çırpın.
6,Sonrasında Stoacı bir hava esti.
7,Su parkı da iki bin altı yılında açıldı.
8,Son bir buçuk iki yıldır açıkça yalan söyledik.
9,Bu cesur iddialar doğru mu?


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/3478 [00:00<?, ?ex/s]

  0%|          | 0/1647 [00:00<?, ?ex/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,i̇nşaat sırbistanda da en tehlikeli meslek
1,bu yöndeki haberler asılsızdır
2,tahsilat firmaları altın çağlarını yaşıyorlar
3,dini muhafazakârlar toplumun yüzde otuz üç virgül beşini kapsıyor
4,terör fazla önem vermediğimiz bir sorun
5,dragulescu kategorinin favorisiydi
6,reform süreci bir kenara bırakıldı
7,o zamandan bu yana bin engelli iş buldu
8,arnavutluk kırk ikinci makedonya ise yetmiş ikinci sırada
9,turnuvaya kırk bir ülkeden toplam üç yüz yetmiş sporcu katıldı


In [ ]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    return batch

In [ ]:
common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)

  0%|          | 0/3478 [00:00<?, ?ex/s]

  0%|          | 0/1647 [00:00<?, ?ex/s]

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 'ç': 27,
 'ë': 28,
 'ö': 29,
 'ü': 30,
 'ğ': 31,
 'ı': 32,
 'ş': 33,
 '̇': 34}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

37

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
repo_name = "wav2vec2-large-xls-r-300m-turkish-colab-2"

In [ ]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='wav2vec2-large-xls-r-300m-turkish-colab-2' as keyword args. From version 0.7 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:676: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2 into local empty directory.
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2
   e3c75b3..5da9c99  main -> main



'https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/commit/5da9c9957b854a59cf28845eb6aec15d7b62a957'

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/05be0c29807a73c9b099873d2f5975dae6d05e9f7d577458a2466ecb9a2b0c6b/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_17346025.mp3'

In [ ]:
common_voice_train[0]["audio"]

{'array': array([0.        , 0.        , 0.        , ..., 0.0002867 , 0.00025517,
        0.00025791], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/05be0c29807a73c9b099873d2f5975dae6d05e9f7d577458a2466ecb9a2b0c6b/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_17346025.mp3',
 'sampling_rate': 48000}

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'array': array([0.        , 0.        , 0.        , ..., 0.0001435 , 0.00015033,
        0.00028025], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/05be0c29807a73c9b099873d2f5975dae6d05e9f7d577458a2466ecb9a2b0c6b/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_17346025.mp3',
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

fakat bu dönüşüm nasıl gerçekleşebilir bu açmazdan nasıl çıkılabilir


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: seçimlerden sonraki duruma ilişkin bilgileriniz ne yönde
Input array shape: (76032,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/3478 [00:00<?, ?ex/s]

  0%|          | 0/1647 [00:00<?, ?ex/s]

# Training

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.weight', 'project_hid.bias', 'quantizer.codevectors', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=2, #initially was 16
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=3,  # initially was 30 
  gradient_checkpointing=True,
  fp16=True,
  save_steps=200,
  eval_steps=100,
  logging_steps=200,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/content/wav2vec2-large-xls-r-300m-turkish-colab-2 is already a clone of https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3478
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 2607


Step,Training Loss,Validation Loss,Wer
100,No log,4.211431,1.000000
200,6.181800,3.253589,1.000000
300,6.181800,3.155296,0.999898
400,3.047400,2.617135,1.000000
500,3.047400,1.471327,0.968747
600,1.650700,1.172816,0.905117
700,1.650700,1.082252,0.898478
800,1.082500,0.912709,0.866408
900,1.082500,0.866293,0.821877
1000,0.870400,0.744221,0.769176


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1647
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1647
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xls-r-300m-turkish-colab-2/checkpoint-200
Configuration saved in wav2vec2-large-xls-r-300m-turkish-colab-2/checkpoint-200/config.json
Model weights saved in wav2vec2-large-xls-r-300m-turkish-colab-2/checkpoint-200/pytorch_model.bin
Feature extractor saved in wav2vec2-large-xls-r-300m-turkish-colab-2/checkpoint-200/preprocessor_c

TrainOutput(global_step=2607, training_loss=1.2891541320243283, metrics={'train_runtime': 20195.1357, 'train_samples_per_second': 0.517, 'train_steps_per_second': 0.129, 'total_flos': 1.263680067538083e+18, 'train_loss': 1.2891541320243283, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-xls-r-300m-turkish-colab-2
Configuration saved in wav2vec2-large-xls-r-300m-turkish-colab-2/config.json
Model weights saved in wav2vec2-large-xls-r-300m-turkish-colab-2/pytorch_model.bin
Feature extractor saved in wav2vec2-large-xls-r-300m-turkish-colab-2/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.18G [00:00<?, ?B/s]

Upload file runs/May16_09-05-42_278279a70826/events.out.tfevents.1652692040.278279a70826.71.2:  22%|##1       …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2
   5da9c99..4fd6019  main -> main

Dropping the following result as it does not have all the necessary fields:
{'dataset': {'name': 'common_voice', 'type': 'common_voice', 'args': 'tr'}}


TypeError: ignored

# Evaluation

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor, Wav2Vec2ForCTC

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2")

https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpwputdocj


Downloading:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

storing https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/96c63f08ce18710bcae81608ca84a7d307d8805c70d6462132f553941d42cb3e.fe08f4413732b6dcecc4a8c1eac32201fd88a95bef47115581452f09c69317e8
creating metadata file for /root/.cache/huggingface/transformers/96c63f08ce18710bcae81608ca84a7d307d8805c70d6462132f553941d42cb3e.fe08f4413732b6dcecc4a8c1eac32201fd88a95bef47115581452f09c69317e8
loading configuration file https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/96c63f08ce18710bcae81608ca84a7d307d8805c70d6462132f553941d42cb3e.fe08f4413732b6dcecc4a8c1eac32201fd88a95bef47115581452f09c69317e8
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-xls-r-300m",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment":

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

storing https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/d9e6fa045f6b05fa59b5853c48dda7c8bd4a2d02928e07b99e9af0b1528ded13.5e0b239c752bccf6507e3f4f795b67cb342cb3213fb8b13930b2e2f954289e71
creating metadata file for /root/.cache/huggingface/transformers/d9e6fa045f6b05fa59b5853c48dda7c8bd4a2d02928e07b99e9af0b1528ded13.5e0b239c752bccf6507e3f4f795b67cb342cb3213fb8b13930b2e2f954289e71
loading weights file https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/d9e6fa045f6b05fa59b5853c48dda7c8bd4a2d02928e07b99e9af0b1528ded13.5e0b239c752bccf6507e3f4f795b67cb342cb3213fb8b13930b2e2f954289e71
All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

All the weights of Wav2Vec2ForCTC were initialized from the model checkpoint at hasanalay/wav2vec2-large-xls-r-300m-turkish-cola

...

storing https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/7bc031e8f846369f8bc3d8732c6e2ed9148840663db34d9553d48ec3043421e3.fe42a20318f479badf9d03df807b41ad17f3420449f46dba147ec081210c9490
creating metadata file for /root/.cache/huggingface/transformers/7bc031e8f846369f8bc3d8732c6e2ed9148840663db34d9553d48ec3043421e3.fe42a20318f479badf9d03df807b41ad17f3420449f46dba147ec081210c9490
https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/added_tokens.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp3tgnbuof


Downloading:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

storing https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/added_tokens.json in cache at /root/.cache/huggingface/transformers/e12ddd6926ddf773a9cf72f45c9957a36dcb60ff7df92260687220ec8d104616.7d7e5e2eda80be65d23eb33796dd7f8ab4abf6a23d76db17e1a72a582bc400f5
creating metadata file for /root/.cache/huggingface/transformers/e12ddd6926ddf773a9cf72f45c9957a36dcb60ff7df92260687220ec8d104616.7d7e5e2eda80be65d23eb33796dd7f8ab4abf6a23d76db17e1a72a582bc400f5
https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpc6_sn93m


Downloading:   0%|          | 0.00/309 [00:00<?, ?B/s]

storing https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/015205d7050c1a90959b1e7057c574326be9ee32be0664ba599248ff5006087d.52ff36a8492158dc5f514fcf8fa507ed5251bf82f2ab8108bbbcdc91ce4f4ac9
creating metadata file for /root/.cache/huggingface/transformers/015205d7050c1a90959b1e7057c574326be9ee32be0664ba599248ff5006087d.52ff36a8492158dc5f514fcf8fa507ed5251bf82f2ab8108bbbcdc91ce4f4ac9
loading file https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/7bc031e8f846369f8bc3d8732c6e2ed9148840663db34d9553d48ec3043421e3.fe42a20318f479badf9d03df807b41ad17f3420449f46dba147ec081210c9490
loading file https://huggingface.co/hasanalay/wav2vec2-large-xls-r-300m-turkish-colab-2/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ce32f6e3cf8d88058992982c2ae5f29a7595

Orijinal modeli kullanamadığım için "dundar/wav2vec2-large-xlsr-53-turkish" bu repodaki modeli kullandım.

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("dundar/wav2vec2-large-xlsr-53-turkish")
model = Wav2Vec2ForCTC.from_pretrained("dundar/wav2vec2-large-xlsr-53-turkish").to("cuda")

In [ ]:
import torch

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
common_voice_test_transcription = load_dataset("common_voice", "tr", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Using custom data configuration tr-24242e0bc11eddb3


Generating train split:   0%|          | 0/1831 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1647 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1647 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/325 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/18685 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/1726 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/tr-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e. Subsequent calls will reuse this data.


In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["sentence"].lower())

Prediction:
pek çoğu da romen toplumundann geliyor

Reference:
pek çoğu da roman toplumundan geliyor.
